In [64]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datasets import load_dataset
from pymatgen.core import Structure
from pymatgen.util.testing import PymatgenTest
from scipy.spatial.distance import jensenshannon

from lematerial_forgebench.benchmarks.distribution_benchmark import (
    DistributionBenchmark,
)
from lematerial_forgebench.metrics.distribution_metrics import (
    MMD,
    FrechetDistance,
    JSDistance,
)
from lematerial_forgebench.preprocess.distribution_preprocess import (
    DistributionPreprocessor,
)
from lematerial_forgebench.utils.distribution_utils import (
    map_space_group_to_crystal_system,
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
with open("lematbulk_properties.pkl", "rb") as f:
    test_lemat = pickle.load(f)

In [9]:
test = PymatgenTest()

filename = "CsBr.cif"
structure = Structure.from_file(filename)
structure = structure.remove_oxidation_states()

filename2 = "CsPbBr3.cif"
structure2 = Structure.from_file(filename2)
structure2 = structure2.remove_oxidation_states()

structures = [
    structure,
    structure2,
    test.get_structure("Si"),
    test.get_structure("LiFePO4"),
]

C:\Users\samue\AppData\Local\Temp\ipykernel_24916\2264363178.py:1: FutureWarning: PymatgenTest is deprecated, and will be removed on 2026-01-01
Use MatSciTest in pymatgen.util.testing instead.
  test = PymatgenTest()


In [76]:
distribution_preprocessor = DistributionPreprocessor()
preprocessor_result = distribution_preprocessor(structures)

In [77]:
test_df = pd.DataFrame(preprocessor_result.processed_structures, columns = ["Volume", "Density(g/cm^3)", "Density(atoms/A^3)", 
                                                                            "SpaceGroup", "CrystalSystem", "CompositionCounts",
                                                                            "Composition"])

In [201]:
benchmark = DistributionBenchmark()
benchmark_result = benchmark.evaluate([test_df], test_lemat.iloc[0:10])

MMD_results


In [200]:
benchmark_result.evaluator_results

{'JSDistance': {'combined_value': None,
  'JSDistance_value': {'SpaceGroup': np.float64(0.8325546111576977),
   'CrystalSystem': np.float64(0.2788095948658411),
   'CompositionCounts': np.float64(0.6901659684588751),
   'Composition': np.float64(0.713320308611451)},
  'metric_results': {'JSDistance': MetricResult(metrics={'Jensen_Shannon_Distance': {'SpaceGroup': np.float64(0.8325546111576977), 'CrystalSystem': np.float64(0.2788095948658411), 'CompositionCounts': np.float64(0.6901659684588751), 'Composition': np.float64(0.713320308611451)}}, primary_metric='Jensen_Shannon_Distance', uncertainties={}, config=DistributionMetricConfig(name='Distribution', description='Measures distance between two reference distributions', lower_is_better=False, n_jobs=1, xy=0.1, xz=False), computation_time=0.006000041961669922, n_structures=1, individual_values=[{'SpaceGroup': np.float64(0.8325546111576977), 'CrystalSystem': np.float64(0.2788095948658411), 'CompositionCounts': np.float64(0.69016596845887

In [205]:
distribution_preprocessor = DistributionPreprocessor()
preprocessor_result = distribution_preprocessor(structures)
test_df = pd.DataFrame(preprocessor_result.processed_structures, columns = ["Volume", "Density(g/cm^3)", "Density(atoms/A^3)", 
                                                                        "SpaceGroup", "CrystalSystem", "CompositionCounts",
                                                                        "Composition"])

with open("lematbulk_properties.pkl", "rb") as f:
    test_lemat = pickle.load(f)
metric = MMD()
result = metric([test_df], test_lemat)

In [206]:
result.metrics

{'MMD': {'Volume': np.float64(0.2545689002257389),
  'Density(g/cm^3)': np.float64(0.41936986825451567),
  'Density(atoms/A^3)': np.float64(4.826848978067844e-06)}}

In [209]:
    assert not np.isnan(result.metrics["MMD"].values())


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [198]:
benchmark.evaluators

{'JSDistance': <lematerial_forgebench.evaluator.MetricEvaluator at 0x12998b53150>,
 'MMD': <lematerial_forgebench.evaluator.MetricEvaluator at 0x12998c35750>}

In [191]:
benchmark_result.evaluator_results["JSDistance"]["JSDistance_value"]

{'SpaceGroup': np.float64(0.8325546111576977),
 'CrystalSystem': np.float64(0.2788095948658411),
 'CompositionCounts': np.float64(0.6901659684588751),
 'Composition': np.float64(0.713320308611451)}

In [192]:
benchmark_result.evaluator_results["MMD"]["MMD_value"]

{'Volume': np.float64(0.3578373686257075),
 'Density(g/cm^3)': np.float64(0.16174021104526248),
 'Density(atoms/A^3)': np.float64(0.0003819190830129937)}